<h1>Short note on the calculation of HMI pixel size</h1>

In this notebook it will be discussed how you can calculate the size and area of each pixel. A short introdutory explanation for HMI coordinates is presented by <a href='http://sun.stanford.edu/~beck/JSOC/HMI_WCS_Dummies.pdf'>J.G. Beck (2010)</a>. A sample data can also be found in this notebook repository but alternatively you can download any <TT>hmi.sharp_cea<TT> data and use it to run the next few lines.

From the <a href='http://jsoc.stanford.edu/doc/keywords/JSOC_Keywords_for_metadata.pdf'> JSOC Keywords for metadata (2013) </a> it is proposed for HMI that it should have a FIXED value for the radius of the Sun in meters combined with a measured average radius (pixels) of the solar image using a non-changing definition of the solar limb, combined with the known distance between the telescope and the solar center (not photosphere). The keywords “<TT>R_SUN</TT>”, “<TT>CRPIX1</TT>”, and “<TT>CRPIX2</TT>” will contain the key information from which the other values are computed. <TT>R_SUN</TT>, <TT>CRPIXj</TT> are all in pixels with center of the lower left pixel of the array set to 1.0, 1.0. The older MDI <TT>X0</TT> and <TT>Y0</TT> are the location of the solar disk center in the image and are $X0 = CRPIX1-1$ and $Y0 = CRPIX2 -1$. <TT>R_SUN</TT> is first determined in level1.0 processing so these keywords are absent prior to level1.0 data. They are included in the level-1 data only for reporting the measured quantities. They are not propagated to higher processing levels. From SDO altitude and orbit information it is defined that:

- <TT>DSUN_OBS</TT>: is the distance to Sun center from spacecraft in meters which is basically $1.5 \times 10^{11} m$;
- <TT>RSUN_REF</TT>: radius of Sun in meters, agreed upon standard ($6.96\times10^{8}$ for HMI) but must be consistent with <TT>WAVELNTH</TT> keyword.
- <TT>R_SUN</TT>: Sun radius computed from the same fit to each image (given in pixels);
- <TT>IM_SCALE</TT>: it is computed from computed from <TT>R_SUN</TT>, <TT>RSUN_REF</TT> and <TT>IM_SCALE</TT> and given in arc-sec per CCD pixel value for the particular instrument. Note that <TT>IM_SCALE</TT> does not change when the image is rebinned on the ground. It is a characteristic of the data as observed. 

From HMI meta data we use <TT>CDELT</TT> which is the coordinate increment at reference point <b>IN ARCSEC</b> (see <a href='https://www.aanda.org/articles/aa/pdf/2002/45/aah3859.pdf'> Greisen and Calabretta 2002 </a>) to calculate the pixel size. For both AIA and HMI, lvl1.0 and above both <TT>CDELT1</TT> and <TT>CDELT2</TT> are equal to or scaled from <TT>IM_SCALE</TT>.

So we calculate the HMI pixel size using:

$$dx = \frac{2\pi\cdot CDELT}{360} \cdot R_{\odot} $$

where the first term, that is multiplying the solar radius $(R_{\odot})$, represents the fraction of the angle covered by each pixel in the image which we then multiply by the solar radius. In Python we can calculate it using the following lines:

In [1]:
path = './hmi*.fits'
import sunpy.map
import numpy as np

# Taking the header information from one of the files.
meta = sunpy.map.Map(path).meta

# Defining a dx in meter based on HMI resolution.
dx = (2*np.pi*meta['CDELT2']*6.955e8/360)

print(f'The HMI pixel size is {dx} meters.')

The HMI pixel size is 364162.94842861686 meters.


One arc second is equal to about 725 kilometers on the Sun and considering that HMI resolution is of 0.5 arc second it should be roughly 362.5 km (if you want to be more precise it is 0.504 arc sec as shown in Figure 9 of <a href='https://link.springer.com/content/pdf/10.1007%2Fs11207-014-0516-8.pdf'>J.T.Hoeksema et.al 2014</a>). Comparing this estimation with the value we actually got calculating from the header we have that:

In [2]:
# Calculating the difference in percentual
(dx/1000-362.5)/100

0.016629484286168575

Which represents a difference of much less than 1% probably induced by HMI orbit itself. Since each pixel is essentially a square the area of it would then be our dx squared, therefore:

In [3]:
area = np.power(dx,2)
print(f'The HMI pixel area is {area} meters squared.')

The HMI pixel area is 132614653008.22346 meters.


Which is basically $1.3\times10^{11}$ m$^{2}$. It is important to remember that in the <TT>CEA</TT> projection each pixel represents the same surface area in the photosphere (pg.18 <a href='https://link.springer.com/content/pdf/10.1007%2Fs11207-014-0516-8.pdf'>J.T.Hoeksema et.al 2014</a>) therefore for this data series it is not necessary to run this calculation for every pixel.